In [ ]:
# importing panda and datetime library 
import pandas as pd
import datetime
import math
import array as arr
#open file

df = pd.read_csv(r'C:\Users\user\Desktop\filename.txt', delimiter = "\t", header=6)
df.head()

# creating the DataFrame 
team = pd.DataFrame(df)
# adding column name to the respective columns 
team.columns =['time', 'values'] 
results = [] #create results list
duration = []
mini = []
hr = []
#measure the number or rows and caclulate the number of 5min session
k= len(df)
duration.append((team['values'][0]))
team['time'] = pd.to_datetime(team['time'])
o = team.iloc[1]['time']
time_change = datetime.timedelta(minutes=5)


for i in range (1,k,1):
    millis=duration[i-1]
    millis = int(millis)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))
    minutes = int(minutes) + 1
    duration.append((team['values'][i] + duration[i-1]))
    mini.append(minutes)
   
je=minutes
for j in range (1,je,5): # step miniutes
    for z in range (1,k-5,1):
   # hr.clear
        x= mini[z]
    #print (x)
        if x == j or x == j+1 or x == j+2 or x == j+3 or x == j+4: #select the 5min step
            hr.append(team['values'][z])
            
    print (hr)


    rr_intervals_list = hr
    
    #https://pypi.org/project/hrv-analysis/
    #Outliers and ectopic beats filtering methods
    from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
    
    # This remove outliers from signal
    rr_intervals_without_outliers = remove_outliers(rr_intervals=rr_intervals_list,  low_rri=200, high_rri=3000)
    
    # This replace outliers nan values with linear interpolation
    interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers, interpolation_method="linear")
    
    # This remove ectopic beats from signal
    nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals, method="malik")
    
    # This replace ectopic beats nan values with linear interpolation
    interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)
    
    
    interpolated_nn_intervals  = list(filter(lambda r: str(r) != 'nan', interpolated_nn_intervals)) #[r if math.isnan(r) else r for r in interpolated_nn_intervals]
    #Features calculation
    print (interpolated_nn_intervals)
    from hrvanalysis import get_time_domain_features
    time_domain_features = get_time_domain_features(interpolated_nn_intervals)
    print(o)
    print(time_domain_features)
   
    
    from hrvanalysis import get_frequency_domain_features
    frequency_domain_features = get_frequency_domain_features(interpolated_nn_intervals)
    print(frequency_domain_features)
       
    results.append([o, time_domain_features, frequency_domain_features])
    o = o + time_change
    hr.clear()    

table = pd.DataFrame(results)
table.to_csv('test.csv', index=False)